In [ ]:
module commutator #(
    parameter WIDTH = 9
)(
    input  wire                  mode,     // 0: switch, 1: bypass
    input                        state1_com1_flag,
    input                        state2_com1_flag,
    input                        state2_com2_flag,
    input                        state3_com1_flag,
    input                        state3_com2_flag,
    input                        state3_com3_flag,
    input  wire signed [WIDTH-1:0] inUI_re,
    input  wire signed [WIDTH-1:0] inUI_im,
    input  wire signed [WIDTH-1:0] inLI_re,
    input  wire signed [WIDTH-1:0] inLI_im,
    output reg  signed [WIDTH-1:0] Up_out_re,
    output reg  signed [WIDTH-1:0] Up_out_im,
    output reg  signed [WIDTH-1:0] Low_out_re,
    output reg  signed [WIDTH-1:0] Low_out_im
);


always @(*) begin
    if (mode) begin  // bypass
        if (!state1_com1_flag) begin
            Up_out_re = inLI_re;
            Up_out_im = inLI_im;
        end else begin
            Low_out_re = inLI_re;
            Low_out_im = inLI_im;
        end
        
    end else begin  // switch
        if(state2_com1_flag) begin
            Up_out_re = inUI_re;
            Up_out_im = inUI_im;
        end else if (state2_com2_flag) begin
            Up_out_re = inLI_re;
            Up_out_im = inLI_im;
            Low_out_re = inUI_re;
            Low_out_im = inUI_im;
        end else begin
        
        end 

        if (state3_com1_flag) begin
            Up_out_re = inUI_re;
            Up_out_im = inUI_im;
            Low_out_re = inLI_re;
            Low_out_im = inLI_im;
        end else if (state3_com2_flag) begin
            Up_out_re = inLI_re;
            Up_out_im = inLI_im;
            Low_out_re = inUI_re;
            Low_out_im = inUI_im;
        end else if (state3_com3_flag) begin
            Low_out_re = inLI_re;
            Low_out_im = inLI_im;
        end

    end
end

endmodule


In [ ]:
reg state1_com1_flag_tmp;

    reg state2_com1_flag_tmp;
    reg state2_com2_flag_tmp;

    reg state3_com1_flag_tmp;
    reg state3_com2_flag_tmp;
    reg state3_com3_flag_tmp;

    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            state2_com1_flag_tmp <= 0;
            state2_com2_flag_tmp <= 0;
            state1_com1_flag_tmp <= 0;
        end else begin
            // Default reset
            if (counter > 15 && counter < 32) begin
                state1_com1_flag_tmp <= 1;
            end else begin
                state1_com1_flag_tmp <= 0;
            end

            if (counter > 15 && counter < 24) begin
                state2_com1_flag_tmp <= 1;
            end else begin
                state2_com1_flag_tmp <= 0;
            end

            if (counter > 23 && counter < 32) begin
                state2_com2_flag_tmp <= 1;
            end else begin
                state2_com2_flag_tmp <= 0;
            end

            if ((counter > 23 && counter < 28) || (counter > 31 && counter < 36)) begin
                state3_com1_flag_tmp <= 1;
            end else begin
                state3_com1_flag_tmp <= 0;
            end
            if ((counter > 27 && counter < 32) || (counter > 35 && counter < 40)) begin
                state3_com2_flag_tmp <= 1;
            end else begin
                state3_com2_flag_tmp <= 0;
            end
            if ((counter > 39 && counter < 44)) begin
                state3_com3_flag_tmp <= 1;
            end else begin
                state3_com3_flag_tmp <= 0;
            end
        end
    end



    
//----------------------------------------------------------
    assign state1_com1_flag = state1_com1_flag_tmp;

    assign state2_com1_flag = state2_com1_flag_tmp;
    assign state2_com2_flag = state2_com2_flag_tmp;

    assign state3_com1_flag = state3_com1_flag_tmp;
    assign state3_com2_flag = state3_com2_flag_tmp;
    assign state3_com3_flag = state3_com3_flag_tmp;

In [ ]:
module fft (
    input clk,
    input rst_n,
    input wire signed [8:0] FFTInRe,
    input wire signed [8:0] FFTInIm,


    output wire signed [8:0] MDCOutUpRe,
    output wire signed [8:0] MDCOutUpIm,
    output wire signed [8:0] MDCOutDownRe,
    output wire signed [8:0] MDCOutDownIm

);
//----------------State1------------------
    reg signed [8:0] state1_inUI_re;
    reg signed [8:0] state1_inUI_im;
    reg signed [8:0] state1_inLI_re;
    reg signed [8:0] state1_inLI_im;

    wire signed [8:0] state1_outUp_re;
    wire signed [8:0] state1_outUp_im;
    wire signed [8:0] state1_outL_re;
    wire signed [8:0] state1_outL_im;

    wire signed [3:0] rom_16_counter;

//----------------State2------------------
    wire signed [8:0] state2_outUp_re;
    wire signed [8:0] state2_outUp_im;
    wire signed [8:0] state2_outL_re;
    wire signed [8:0] state2_outL_im;

    wire signed [2:0] rom_8_counter;

//----------------State3------------------
    wire signed [8:0] state3_outUp_re;
    wire signed [8:0] state3_outUp_im;
    wire signed [8:0] state3_outL_re;
    wire signed [8:0] state3_outL_im;

    wire signed [1:0] rom_4_counter;


    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            state1_inUI_re <= 0;
            state1_inUI_im <= 0;
            
            
        end else begin
            state1_inLI_re <= FFTInRe;
            state1_inLI_im <= FFTInIm;
        end
    end

    wire [6:0] state_code;

    controller controler_1(clk, 
                        rst_n, 
                        rom_16_counter, 
                        rom_8_counter, 
                        rom_4_counter,
                        state_code
    );
    
    //---------State1-------------

    fft_state1 state1(clk, 
                    rst_n, 
                    state_code,
                    rom_16_counter, 
                    state1_inUI_re, 
                    state1_inUI_im, 
                    state1_inLI_re, 
                    state1_inLI_im, 
                    state1_outUp_re, 
                    state1_outUp_im, 
                    state1_outL_re, 
                    state1_outL_im
    );

    //---------State2-------------


    fft_state2#(9) state2(clk, 
                        rst_n, 
                        state_code,
                        rom_8_counter, 
                        state1_outUp_re, 
                        state1_outUp_im, 
                        state1_outL_re, 
                        state1_outL_im, 
                        state2_outUp_re, 
                        state2_outUp_im, 
                        state2_outL_re, 
                        state2_outL_im
    );

    //------------State3---------
    fft_state3#(9) state3(clk, 
                        rst_n, 
                        state_code,
                        rom_4_counter, 
                        state2_outUp_re, 
                        state2_outUp_im, 
                        state2_outL_re, 
                        state2_outL_im, 
                        state3_outUp_re, 
                        state3_outUp_im, 
                        state3_outL_re, 
                        state3_outL_im
    );


    assign MDCOutUpRe = state3_outUp_re;
    assign MDCOutUpIm = state3_outUp_im;
    assign MDCOutDownRe = state3_outL_re;
    assign MDCOutDownIm = state3_outL_im;


endmodule